I have created this project to generate and test Python code based on natural language queries. It uses Microsoft's AutoGen framework to manage two agents: one for generating code and another for executing and validating it. The system leverages LM Studio to host Hugging Face models like Llama-2 locally, ensuring fast and secure processing.The main workflow includes 2 agents interacting with each other, generating(llama-3.2-1b-instruct) and debugging code. One agent uses Llm to generate code from natural language and other executes it and gives feedback (user can also give feedback) to the code writer generator and this conversation moves accordingly. The generated code is and all outputs, are saved in a text file for reference.
To set up, the project requires installing the AutoGen framework, configuring LM Studio, and hosting the desired model locally. Users can run the system via a Jupyter notebook or the main script.
Moreover, 2 use case is included with all the interactions in usecase directory.

In [1]:
import autogen
from autogen import ConversableAgent
import re
from typing import Any, Dict, List, Union
from pydantic import BaseModel

In [2]:
# creating configuration for using llm for writing code
config_list = autogen.config_list_from_json(env_or_file="config.json")

In [3]:
# Create an agent with code executor configuration
code_executor_agent = ConversableAgent(
    "code_executor_agent",
    llm_config=False,  # Turn off LLM for this agent.
    code_execution_config={"use_docker":False},
    human_input_mode="ALWAYS",  # Always take human input for this agent for safety
    )

In [4]:
# prompt for the code writer agent
code_writer_system_message = """You are a helpful AI assistant.Solve tasks using your coding and language skills.
    In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
    Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
    When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
    If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
    When user sends this "exitcode: 0 (execution succeeded)", reply only with "TERMINATE".
    If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
    When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
    """

# Create an agent with LLM configuration for writing code
code_writer_agent = ConversableAgent(
     "code_writer_agent",
     system_message=code_writer_system_message, # prompt for the code writer agent
     llm_config={"config_list":config_list},
     code_execution_config=False,  # Turn off code execution for this agent.
     )

In [5]:
class CodeBlock(BaseModel):
    """A class that represents a code block."""

    """The code to execute."""
    code: str

    """The language of the code."""
    language: str

In [6]:
CODE_BLOCK_PATTERN = r"```[ \t]*(\w+)?[ \t]*\r?\n(.*?)\r?\n[ \t]*```" # pattern to extract code blocks from a message

In [7]:
class MarkdownCodeExtractor:
    """A class that extracts code blocks from a message using Markdown syntax."""
    
    def content_str(self,content: Union[str, List[Dict[str, Any]], None]) -> str:
        """Converts `content` into a string format.
        This function processes content that may be a string, a list of dictionary of string or None,
        and converts it into a string. Text is directly appended to the result string.
        If the content is None, an empty string is returned.
        
        Args:
        - content (Union[str, List, None]): The content to be processed. Can be a string, a list of dictionaries
        representing text or None.
        
        Returns:
        str: A string representation of the input content.
    """
        
        if content is None:
            return "" # return empty string if no content
        
        if isinstance(content, str):
            return content # return content if it is a string
        
        if not isinstance(content, list):
            raise TypeError(f"content must be None, str, or list, but got {type(content)}")
        
        rst = "" # string representing the content

        for item in content:
            if not isinstance(item, dict):
                raise TypeError("Wrong content format: every element should be dict if the content is a list.")
            assert "content" in item, "Wrong content format. Missing 'content' key in content's dict."
            rst += item["content"] # append the content to the result string
        return rst
        
    def infer_lang(self,code: str) -> str:
        """infer the language for the code.
        """
        if code.startswith("python ") or code.startswith("pip") or code.startswith("python3 "):
            return "sh"
        # check if code is a valid python code
        try:
            compile(code, "test", "exec")
            return "python"
        except SyntaxError:
            # not a valid python code
            return "unknown"

    def extract_code_blocks(self, message: Union[str, List[Dict[str, Any]], None]) -> List[CodeBlock]:
        """Extract code blocks from a message. If no code blocks are found,
        return an empty list.

        Args:
            message (str): The message to extract code blocks from.

        Returns:
            List[CodeBlock]: The extracted code blocks or an empty list.
        """

        text = self.content_str(message) # convert the message to a string

        match = re.findall(CODE_BLOCK_PATTERN, text, flags=re.DOTALL) # find all code blocks in the message
        if not match:
            return [] # return empty list if no code blocks are found
        
        code_blocks = []
        for lang, code in match:
            if lang == "":
                lang = self.infer_lang(code) # infer the language of the code
            if lang == "unknown":
                lang = ""
            code_blocks.append(CodeBlock(code=code, language=lang)) # append the code block to the list of code blocks
        return code_blocks

In [ ]:
message= input("Hello! How can I assist you with your code today?") # input from user
    
# Initiate chat between the two agents.
chat_result = code_executor_agent.initiate_chat(
    code_writer_agent,
    message=message,
    )

In [9]:
# Extract code blocks from the chat history
code_extracter=MarkdownCodeExtractor()
code=code_extracter.extract_code_blocks(chat_result.chat_history)

In [10]:
# Write the code to a file.
file = open(f"{message}.txt", "w")
file.write(message+"\n")
for i in code:
     file.write(i.code+"\n")   
file.close() 